# Deep Research APIの紹介

## 背景

Deep Research APIは、推論、計画、および実世界の情報全体にわたる統合を必要とする複雑な研究ワークフローを自動化することを可能にします。これは、高レベルなクエリを受け取り、タスクを分解し、ウェブ検索を実行し、結果を統合することができるエージェント型モデルを活用して、構造化された引用豊富なレポートを返すように設計されています。

このプロセスが抽象化されているChatGPTとは異なり、このAPIは直接的なプログラマティックアクセスを提供します。リクエストを送信すると、モデルは自律的にサブクエスチョンを計画し、ウェブ検索やコード実行などのツールを使用し、最終的な構造化された応答を生成します。このクックブックでは、Deep Research APIとその使用方法について簡潔に紹介します。

以下のモデルを使用して`responses`エンドポイント経由でDeep Researchにアクセスできます：

- `o3-deep-research-2025-06-26`: 詳細な統合と高品質な出力に最適化
- `o4-mini-deep-research-2025-06-26`: 軽量で高速、レイテンシに敏感なユースケースに最適

## セットアップ

### 必要なパッケージのインストール
OpenAI Python SDKの最新バージョンをインストールしてください。

In [ ]:
!pip install --upgrade openai

### 認証
OpenAIクライアントをインポートし、APIキーで初期化します。

In [ ]:
from openai import OpenAI
OPENAI_API_KEY="" # YOUR OPENAI_API_KEY
#OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

## はじめに

Deep Research APIコールの例を見ていきましょう。ヘルスケア金融サービス企業で働いており、2型糖尿病と肥満の治療に使用される最近の薬物、特にセマグルチドの経済的影響に関する詳細なレポートを作成する任務を負っているとします。私たちの目標は、臨床結果、費用対効果、地域別価格データを、投資、支払者戦略、または政策提言に役立つ構造化された引用付きの分析に統合することです。

開始するために、以下を行いましょう：
- システムメッセージに私たちの役割を設定し、生成したいレポートの種類を概説する
- 最適な要約を得るため、summaryパラメータを今のところ"auto"に設定する（より詳細なレポートが必要な場合は、summaryを"detailed"に設定できます）
- 必須ツールである`web_search_preview`を含め、オプションで`code_interpreter`を追加する
- backgroundパラメータをTrueに設定する。Deep Researchタスクは実行に数分かかる場合があるため、バックグラウンドモードを有効にすることで、タイムアウトやその他の接続問題を心配することなく、リクエストを非同期で実行できます。

In [ ]:
system_message = """
You are a professional researcher preparing a structured, data-driven report on behalf of a global health economics team. Your task is to analyze the health question the user poses.

Do:
- Focus on data-rich insights: include specific figures, trends, statistics, and measurable outcomes (e.g., reduction in hospitalization costs, market size, pricing trends, payer adoption).
- When appropriate, summarize data in a way that could be turned into charts or tables, and call this out in the response (e.g., “this would work well as a bar chart comparing per-patient costs across regions”).
- Prioritize reliable, up-to-date sources: peer-reviewed research, health organizations (e.g., WHO, CDC), regulatory agencies, or pharmaceutical earnings reports.
- Include inline citations and return all source metadata.

Be analytical, avoid generalities, and ensure that each section supports data-backed reasoning that could inform healthcare policy or financial modeling.
"""

user_query = "Research the economic impact of semaglutide on global healthcare systems."

response = client.responses.create(
  model="o3-deep-research",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": system_message,
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": user_query,
        }
      ]
    }
  ],
  reasoning={
    "summary": "auto"
  },
  tools=[
    {
      "type": "web_search_preview"
    },
    {
      "type": "code_interpreter",
      "container": {
        "type": "auto",
        "file_ids": []
      }
    }
  ]
)

## レスポンスの解析

Deep Research APIのレスポンスには、インライン引用、推論ステップの要約、ソースメタデータとともに、構造化された最終回答が含まれています。

### 最終レポート出力の抽出

これがこのレポートのメインテキスト出力です。

In [ ]:
# Access the final report from the response object
print(response.output[-1].content[0].text)

### インライン引用とメタデータへのアクセス
レスポンステキスト内のインライン引用は注釈が付けられ、対応するソースメタデータにリンクされています。各注釈には以下が含まれます：
- start_index と end_index: 引用が参照するテキスト内の文字範囲
- title: ソースの簡潔なタイトル
- url: 完全なソースURL

この構造により、引用リストや参考文献の作成、下流アプリケーションでのクリック可能なハイパーリンクの追加、レポート内のデータに裏付けられた主張のハイライトと追跡が可能になります。

In [ ]:
annotations = response.output[-1].content[0].annotations
for i, citation in enumerate(annotations):
    print(f"Citation {i+1}:")
    print(f"  Title: {citation.title}")
    print(f"  URL: {citation.url}")
    print(f"  Location: chars {citation.start_index}–{citation.end_index}")


### 中間ステップの検査
Deep Research APIは、推論ステップ、ウェブ検索の呼び出し、コード実行など、エージェントが実行したすべての中間ステップも公開します。これらを使用して、最終的な回答がどのように構築されたかをデバッグ、分析、または視覚化することができます。
各中間ステップは`response.output`に格納され、`type`フィールドがその種類を示します。

#### 推論ステップ
これらは、モデルがサブクエスチョンを通じて推論する際に生成される内部的な要約や計画を表します。

In [ ]:
# Find the first reasoning step
reasoning = next(item for item in response.output if item.type == "reasoning")
for s in reasoning.summary:
    print(s.text)

#### Web検索呼び出し
これらは実行された検索クエリを表示し、モデルが取得した情報を追跡するのに役立ちます。

In [ ]:
# Find the first web search step
search = next(item for item in response.output if item.type == "web_search_call")
print("Query:", search.action["query"])
print("Status:", search.status)

#### コード実行
モデルがコードインタープリター（データの解析やチャートの生成など）を使用した場合、それらのステップは"code_interpreter_call"またはそれに類似するタイプとして表示されます。

In [ ]:
# Find a code execution step (if any)
code_step = next((item for item in response.output if item.type == "code_interpreter_call"), None)
if code_step:
    print(code_step.input)
    print(code_step.output)
else:
    print("No code execution steps found.")


#### Model Context Protocol (MCP)

Deep Researchタスクの一部として独自の内部文書を取り込みたい場合を考えてみましょう。Deep ResearchモデルとResponses APIの両方がMCPベースのツールをサポートしているため、プライベートな知識ストアや他のサードパーティサービスにクエリを実行するように拡張することができます。

以下の例では、Deep Researchが組織の内部セマグルチド研究をオンデマンドで取得できるMCPツールを設定しています。MCPサーバーは、アップロードされたファイルを自動的にベクトル化して高性能な検索を可能にするOpenAI File Storageサービスのプロキシとして機能します。

このシンプルなMCPサーバーを_どのように_構築したかを確認したい場合は、[この関連クックブック](https://cookbook.openai.com/examples/deep_research_api/how_to_build_a_deep_research_mcp_server/readme)を参照してください。

In [ ]:
# system_message includes reference to internal file lookups for MCP.
system_message = """
You are a professional researcher preparing a structured, data-driven report on behalf of a global health economics team. Your task is to analyze the health question the user poses.

Do:
- Focus on data-rich insights: include specific figures, trends, statistics, and measurable outcomes (e.g., reduction in hospitalization costs, market size, pricing trends, payer adoption).
- When appropriate, summarize data in a way that could be turned into charts or tables, and call this out in the response (e.g., “this would work well as a bar chart comparing per-patient costs across regions”).
- Prioritize reliable, up-to-date sources: peer-reviewed research, health organizations (e.g., WHO, CDC), regulatory agencies, or pharmaceutical earnings reports.
- Include an internal file lookup tool to retrieve information from our own internal data sources. If you’ve already retrieved a file, do not call fetch again for that same file. Prioritize inclusion of that data.
- Include inline citations and return all source metadata.

Be analytical, avoid generalities, and ensure that each section supports data-backed reasoning that could inform healthcare policy or financial modeling.
"""

user_query = "Research the economic impact of semaglutide on global healthcare systems."

response = client.responses.create(
  model="o3-deep-research-2025-06-26",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": system_message,
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": user_query,
        }
      ]
    }
  ],
  reasoning={
    "summary": "auto"
  },
  tools=[
    {
      "type": "web_search_preview"
    },
    { # ADD MCP TOOL SUPPORT
      "type": "mcp",
      "server_label": "internal_file_lookup",
      "server_url": "https://<your_mcp_server>/sse/", # Update to the location of *your* MCP server
      "require_approval": "never"
    }
  ]
)


## レスポンスの確認

研究レポートの最初の100文字、その後に引用とMCPツール呼び出しが続きます。

In [ ]:
# Grab the full report text once
report_text = response.output[-1].content[0].text

print("REPORT EXCERPT:")
print(report_text[:100])   # first 100 chars
print("--------")

annotations = response.output[-1].content[0].annotations
target_url = "https://platform.openai.com/storage/files"

for citation in annotations:
    if citation.url.startswith(target_url):
        start, end = citation.start_index, citation.end_index

        # extract exactly the cited span
        excerpt       = report_text[start:end]

        # extract up to 100 chars immediately before the citation
        pre_start     = max(0, start - 100)
        preceding_txt = report_text[pre_start:start]

        print("MCP CITATION SAMPLE:")
        print(f"  Title:       {citation.title}")
        print(f"  URL:         {citation.url}")
        print(f"  Location:    chars {start}–{end}")
        print(f"  Preceding:   {preceding_txt!r}")
        print(f"  Excerpt:     {excerpt!r}")
        break

print("--------")


# EXAMPLE MCP CITATION

# REPORT EXCERPT:
# # Introduction
# Semaglutide – a glucagon-like peptide-1 (GLP-1) analogue – has rapidly become a blo
# --------
# MCP CITATION SAMPLE:
#   Title:       Document file-WqbCdYNqNzGuFfCAeWyZfp
#   URL:         https://platform.openai.com/storage/files/file-WqbCdYNqNzGuFfCAeWyZfp
#   Location:    chars 237–331
#   Preceding:   'and obesity due to its potent clinical efficacy (often inducing ~10–15% body weight loss in trials) '
#   Excerpt:     '([platform.openai.com](https://platform.openai.com/storage/files/file-WqbCdYNqNzGuFfCAeWyZfp))'


# print the MCP tool calls
calls = [
    (item.name, item.server_label, item.arguments)
    for item in response.output
    if item.type == "mcp_call" and item.arguments
]
for name, server, args in calls:
    print(f"{name}@{server} → {args}")
print("--------")

## ChatGPTとDeep Research APIにおける明確化質問

ChatGPTでDeep Researchを使用したことがある場合、クエリを送信した後にフォローアップの質問をすることがよくあることに気づいたかもしれません。これは意図的なものです：ChatGPTは中間モデル（`gpt-4.1`など）を使用して、研究プロセスが開始される前にあなたの意図を明確化し、より多くのコンテキスト（好み、目標、制約など）を収集します。この追加ステップにより、システムはウェブ検索を調整し、より関連性が高く的確な結果を返すことができます。

対照的に、Deep Research APIはこの明確化ステップをスキップします。開発者として、モデルは最初から完全に形成されたプロンプトを期待し、追加のコンテキストを求めたり不足している情報を補完したりしないため、ユーザープロンプトを書き換えたり明確化質問のセットを尋ねたりするこの処理ステップを設定することができます。モデルは単に受け取った入力に基づいて研究を開始します。

APIから強力で信頼性の高い出力を得るために、2つのアプローチを使用できます。
- 別の軽量モデル（例：`gpt-4.1`）を使用してプロンプトリライターを使い、研究モデルに渡す前にユーザークエリを拡張または具体化する
- すべての関連詳細を含める：希望する範囲、比較、指標、地域、優先ソース、期待される出力形式

この設定により、開発者は研究タスクがどのように組み立てられるかを完全にコントロールできますが、入力プロンプトの品質に対してより大きな責任も負うことになります。以下は、後続のdeep researchクエリをより適切に指示するための一般的な`rewriting_prompt`の例です。

![../../images/intro_dr.png](../../../images/intro_dr.png)

以下は書き換えプロンプトの例です：

In [ ]:
suggested_rewriting_prompt = """
You will be given a research task by a user. Your job is to produce a set of instructions for a researcher that will complete the task. Do NOT complete the task yourself, just provide instructions on how to complete it.

GUIDELINES:
1. **Maximize Specificity and Detail**
- Include all known user preferences and explicitly list key attributes or dimensions to consider.
- It is of utmost importance that all details from the user are included in the instructions.

2. **Fill in Unstated But Necessary Dimensions as Open-Ended**
- If certain attributes are essential for a meaningful output but the user has not provided them, explicitly state that they are open-ended or default to no specific constraint.

3. **Avoid Unwarranted Assumptions**
- If the user has not provided a particular detail, do not invent one.
- Instead, state the lack of specification and guide the researcher to treat it as flexible or accept all possible options.

4. **Use the First Person**
- Phrase the request from the perspective of the user.

5. **Tables**
- If you determine that including a table will help illustrate, organize, or enhance the information in the research output, you must explicitly request that the researcher provide them.
Examples:
- Product Comparison (Consumer): When comparing different smartphone models, request a table listing each model's features, price, and consumer ratings side-by-side.
- Project Tracking (Work): When outlining project deliverables, create a table showing tasks, deadlines, responsible team members, and status updates.
- Budget Planning (Consumer): When creating a personal or household budget, request a table detailing income sources, monthly expenses, and savings goals.
Competitor Analysis (Work): When evaluating competitor products, request a table with key metrics, such as market share, pricing, and main differentiators.

6. **Headers and Formatting**
- You should include the expected output format in the prompt.
- If the user is asking for content that would be best returned in a structured format (e.g. a report, plan, etc.), ask the researcher to format as a report with the appropriate headers and formatting that ensures clarity and structure.

7. **Language**
- If the user input is in a language other than English, tell the researcher to respond in this language, unless the user query explicitly asks for the response in a different language.

8. **Sources**
- If specific sources should be prioritized, specify them in the prompt.
- For product and travel research, prefer linking directly to official or primary websites (e.g., official brand sites, manufacturer pages, or reputable e-commerce platforms like Amazon for user reviews) rather than aggregator sites or SEO-heavy blogs.
- For academic or scientific queries, prefer linking directly to the original paper or official journal publication rather than survey papers or secondary summaries.
- If the query is in a specific language, prioritize sources published in that language.
"""

In [ ]:
response = client.responses.create(
    instructions=suggested_rewriting_prompt,
    model="gpt-4.1-2025-04-14",
    input="help me plan a trip to france",
)

In [ ]:
new_query = response.output[0].content[0].text
print(new_query)

この例では、ユーザーが旅行日程、目的地の希望、予算、興味、旅行仲間などの重要な詳細を指定せずに、一般的または漠然とした質問を送信しました。リライトプロンプトがクエリを書き直したため、Deep Researchは一般的な使用例を想定した幅広く包括的な回答を生成しようとします。

この動作は幅広い選択肢を提示するのに役立つ場合がありますが、モデルが多くの可能なシナリオを考慮する必要があるため、冗長性、高いレイテンシ、トークン使用量の増加につながることがよくあります。これは、複雑な計画や統合タスクを引き起こすクエリ（例：複数目的地の旅行日程、比較研究、製品選択）において特に顕著です。

幅広い研究計画をすぐに進める代わりに、完全な回答を生成する前に軽量なモデルを使用してユーザーから穏やかに明確化の質問をし、その後リライトプロンプトを使用してモデルのより明確な出力を得ることを試してみましょう。

In [ ]:
suggested_clariying_prompt = """"
You will be given a research task by a user. Your job is NOT to complete the task yet, but instead to ask clarifying questions that would help you or another researcher produce a more specific, efficient, and relevant answer.

GUIDELINES:
1. **Maximize Relevance**
- Ask questions that are *directly necessary* to scope the research output.
- Consider what information would change the structure, depth, or direction of the answer.

2. **Surface Missing but Critical Dimensions**
- Identify essential attributes that were not specified in the user’s request (e.g., preferences, time frame, budget, audience).
- Ask about each one *explicitly*, even if it feels obvious or typical.

3. **Do Not Invent Preferences**
- If the user did not mention a preference, *do not assume it*. Ask about it clearly and neutrally.

4. **Use the First Person**
- Phrase your questions from the perspective of the assistant or researcher talking to the user (e.g., “Could you clarify...” or “Do you have a preference for...”)

5. **Use a Bulleted List if Multiple Questions**
- If there are multiple open questions, list them clearly in bullet format for readability.

6. **Avoid Overasking**
- Prioritize the 3–6 questions that would most reduce ambiguity or scope creep. You don’t need to ask *everything*, just the most pivotal unknowns.

7. **Include Examples Where Helpful**
- If asking about preferences (e.g., travel style, report format), briefly list examples to help the user answer.

8. **Format for Conversational Use**
- The output should sound helpful and conversational—not like a form. Aim for a natural tone while still being precise.
"""


response = client.responses.create(
    instructions=suggested_clariying_prompt,
    model="gpt-4.1-2025-04-14",
    input="help me plan a trip to france",
)

new_query = response.output[0].content[0].text
print(new_query)

In [ ]:
user_follow_up = """I'd like to travel in August. I'd like to visit Paria and Nice. I'd like to keep it under $1500 for a 7 day trip without including flights.
I'm going with my friend. we're both in our mid-twenties. i like history, really good french food and wine, and hiking
"""
instructions_for_DR = client.responses.create(
    instructions=suggested_rewriting_prompt,
    model="gpt-4.1-2025-04-14",
    input=user_follow_up,
)
instructions_for_deep_research = instructions_for_DR.output[0].content[0].text
print(instructions_for_deep_research)

In [ ]:
deep_research_call = client.responses.create(
  model="o4-mini-deep-research-2025-06-26",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": instructions_for_deep_research,
        }
      ]
    },
  ],
  reasoning={
    "summary": "auto"
  },
  tools=[
    {
      "type": "web_search_preview"
    },
  ]
)

In [ ]:
# Access the final report from the response object
print(deep_research_call.output[-1].content[0].text)

これで完了です！フランス旅行に向けた詳細な調査レポートが完成しました！

このノートブックでは、Deep Research APIを使用して複雑で実用的な調査タスクを自動化する方法を探求しました。セマグルチドの経済的影響の分析から、あなたに合ったフランス旅行の計画まで幅広く対応しました。Deep Researchは、実世界の証拠に基づいた構造化された引用付きの回答が必要な場合に威力を発揮します。特に優れた使用例には以下があります：
- 製品比較と市場分析
- 競合情報と戦略レポート
- 技術文献レビューと政策統合

調査エージェントの構築、構造化レポートの生成、または高品質な統合をワークフローに組み込むことをお考えの場合、ここでの例がスタートの助けになることを願っています。

次は何でしょうか？[Deep Research Agents](https://cookbook.openai.com/examples/deep_research_api/introduction_to_deep_research_api_agents)